## 7. Comparative LCA

The original notebook of such a comparative static LCA can be found [here](https://nbviewer.jupyter.org/github/PascalLesage/Shared-BW2-notebooks/blob/master/Comparative%20static%20LCA%20in%20Brightway2.ipynb). In solidarity with lactose-intolerant Brightway users, we will now look at **bananas instead of dairy**. So let's see which activities we have in Ecoinvent:

In [ ]:
bananas_unsorted = [act for act in eidb if "banana" in act["name"]]
bananas = sorted(bananas_unsorted, key=lambda k: k["name"])
bananas

['banana production' (kilogram, IN, None),
 'banana production' (kilogram, RoW, None),
 'banana production' (kilogram, EC, None),
 'banana production' (kilogram, CO, None),
 'banana production' (kilogram, CR, None),
 'market for banana' (kilogram, GLO, None)]

Of these activities, we now select the banana production processes for Colombia, Costa Rica, and India, and we select three impact methods on climate change, land use, and water stress.

In [ ]:
banana_CO

'banana production' (kilogram, CO, None)

In [ ]:
[m for m in bd.methods if "water" in str(m)]

In [ ]:
banana_CO = [
    act for act in eidb if "banana" in act["name"] and "CO" in act["location"]
][0]
banana_CR = [
    act for act in eidb if "banana" in act["name"] and "CR" in act["location"]
][0]
banana_IN = [
    act for act in eidb if "banana" in act["name"] and "IN" in act["location"]
][0]

inventory = [banana_CO, banana_CR, banana_IN]

methods = [
    [
        m
        for m in bd.methods
        if "ReCiPe 2016 v1.03, midpoint (H) no LT" in str(m) and "climate change" in str(m)
    ][0],
    [
        m
        for m in bd.methods
        if "ReCiPe 2016 v1.03, midpoint (H) no LT" in str(m)
        and "agricultural land occupation" in str(m)
        and not "w/o LT" in str(m)
    ][0],
    [
        m
        for m in bd.methods
        if "ReCiPe 2016 v1.03, midpoint (H) no LT" in str(m)
        and "water consumption potential (WCP) no LT" in str(m)
    ][0],
]

print("Let's compare\n{},\n{}, and\n{}".format(banana_CO, banana_CR, banana_IN))

Let's compare
'banana production' (kilogram, CO, None),
'banana production' (kilogram, CR, None), and
'banana production' (kilogram, IN, None)


EVERYTHING BELOW NEEDS TO BE UPDATED BASED ON NEW BRIGHTWAY25 SYNTAX!

In [ ]:
results = []


my_lca = bc.LCA(demand=my_functional_unit, data_objs=data_objs)
my_lca.lci()
my_lca.lcia()
my_lca.score

for banana in inventory:
    functional_unit, data_objs, _ = bd.prepare_lca_inputs(
        {banana: 1},
        method=ef_gwp_key,
    )
    lca = bw.LCA({banana: 1})
    lca.lci()
    for method in methods:
        lca.switch_method(method)
        lca.lcia()
        results.append(
            (
                banana["name"],
                banana["location"],
                method[1].title(),
                lca.score,
                bw.methods.get(method).get("unit"),
            )
        )
results

We should probably present these results in a nicer form. Let's use `pandas` for that:

In [ ]:
results_df = pd.DataFrame(
    results, columns=["Name", "Location", "Method", "Score", "Unit"]
)
results_df = pd.pivot_table(
    results_df, index=["Name", "Location"], columns=["Method", "Unit"], values="Score"
)
results_df

In [ ]:
df = pd.DataFrame.from_dict(results).T
df

We can also normalise these results, which may help to get a better overview:

In [ ]:
NormResults_df = results_df / results_df.max()
NormResults_df

Sidenote: We can certainly run this kind of comparative LCA with our own product systems, e.g. the bottle production from above. We would just have to select the respective reference flow...

A faster way of running comparative LCAs is through calculation setups. So, let's get familiar with them! This can be done best by just employing random activities as funcitonal units and selecting the LCIA methods which can be applied to the present biosphere flows (only a subset of these ones). Due to computation time, we decide to use the FORWAST database instead of ecoinvent - although this certainly works, too.

In [ ]:
# define the functional units and LCIA methods; taken from example notebook
functional_units = [{fw.random(): 1} for _ in range(20)]

import random

all_forwast_flows = {exc.input for ds in fw for exc in ds.biosphere()}
suitable_methods = [
    method
    for method in bw.methods
    if {cf[0] for cf in Method(method).load()}.intersection(all_forwast_flows)
]

print("Can use {} of {} LCIA methods".format(len(suitable_methods), len(bw.methods)))
chosen_methods = random.sample(suitable_methods, 8)

In [ ]:
functional_units

In [ ]:
chosen_methods

Now we come to the actual calculation setup which gets defined through our functional units and the chosen LCIA methods:

In [ ]:
my_calculation_setup = {"inv": functional_units, "ia": chosen_methods}

In [ ]:
calculation_setups["set of calculation setups"] = my_calculation_setup

In [ ]:
mlca = MultiLCA("set of calculation setups")
mlca.results

Hm, does not seem to be so difficult. Therefore, let's try it on a better chosen example.

Let's decide on our LCIA method selection:

In [ ]:
[
    m
    for m in bw.methods
    if "ReCiPe" in str(m) and "Midpoint (H)" in str(m) and "land" in str(m)
][:3]

Now a different way fo setting up our list of selected LCIA methods:

In [ ]:
land_methods = [
    m
    for m in bw.methods
    if "ReCiPe" in str(m) and "Midpoint (H)" in str(m) and "land" in str(m)
][:3]
methods = land_methods + [
    [
        m
        for m in bw.methods
        if "ReCiPe" in str(m)
        and "Midpoint (H)" in str(m)
        and "climate change" in str(m)
    ][0],
    [
        m
        for m in bw.methods
        if "ReCiPe" in str(m)
        and "Midpoint (H)" in str(m)
        and "water depletion" in str(m)
    ][0],
]
print(methods)

And now we calculate the results for one functional unit (our banana production from above) and multiple LCIA methods:

**One functional unit, multiple impact categories**

In [ ]:
all_scores = {}
banana_lca = bw.LCA({banana_CO: 1}, methods[0])
banana_lca.lci()
banana_lca.lcia()
for category in methods:
    banana_lca.switch_method(category)
    banana_lca.lcia()
    all_scores[category] = {}
    all_scores[category]["score"] = banana_lca.score
    all_scores[category]["unit"] = bw.Method(category).metadata["unit"]
    print(
        "The score is {:f} {} for impact category {}".format(
            banana_lca.score,
            bw.Method(category).metadata["unit"],
            bw.Method(category).name,
        )
    )

We can certainly present these results also in a nice table:

In [ ]:
df = pd.DataFrame.from_dict(all_scores).T
df

And now let's visualise the results only for the scores that have the same unit, i.e. in this example agricultural and urban land occupation.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
df_m2 = df[df["unit"] == "square meter-year"]
df_m2

In [ ]:
df_m2.plot(kind="barh")

And of course we can find the relative contributions for our selected functional unit, too:

In [ ]:
banana_lca_unitProcessContribution = banana_lca.characterized_inventory.sum(axis=0).A1
banana_lca_unitProcessRelativeContribution = (
    banana_lca_unitProcessContribution / banana_lca.score
)
banana_lca_unitProcessRelativeContribution

And now we calculate the results for multiple functional units and multiple LCIA methods:

**Multiple functional units, multiple impact categories**

In [ ]:
# Set up the production system
prod_sys = [{banana_CO.key: 1}, {banana_CR.key: 1}, {banana_IN.key: 1}]
prod_sys

In [ ]:
bw.calculation_setups["multiLCA"] = {"inv": prod_sys, "ia": methods}
bw.calculation_setups["multiLCA"]
myMultiLCA = bw.MultiLCA("multiLCA")
myMultiLCA.results

df_impact = pd.DataFrame(data=myMultiLCA.results, columns=methods)
df_impact

Just in case we cannot remember for some reason our functional units, we can easily find their definition again:

In [ ]:
myMultiLCA.func_units

The same certainly goes for our chosen LCIA methods:

In [ ]:
myMultiLCA.methods